# DataFrame

Spark 1.0 provided RDDs as its sole data API, supporting transformations and actions that can be used to control how the data is processed in a pipeline. The main advantage of RDDs is that they are simple and well understood because they deal with concrete classes, providing a familiar object-oriented programming style. 

The main disadvantage to RDDs is that they don’t perform particularly well. RDDs serialize all individual object which causes overhead. Other disadvantages are that the code that uses RDD can become less clear in the absense of concrete labels for columns, the use of RDDs demands lower level coding compared to more declarative languages (e.g. SQL), and external libraries rarely provide direct support for RDDs.

In Spark 1.3 DataFrames are supported, which allow for faster processing by allowing objects to be kept in memory instead of serializing everything to disk for every step. DataFrames are the data structure that is used by the very well supported `pandas` library, allowing to use external libraries that support pandas DataFrame in Spark. DataFrames provide a more table-like perspective on data, that can be operated with its own API or declarative languages such as SQL. The engine translates SQL queries to DataFrame operations, therefore, most SQL queries are as efficient as their DataFrame equivalent. 

In [ ]:
data = sc.parallelize([('M', 25), 
                       ('M', 20), 
                       ('M', 30), 
                       ('F', 25),
                       ('F', 20),
                       ('M', 30)])

Creating a DataFrame requires a dataset (e.g. RDD, List), and to provide a schema. In the most simple case, Spark will infer the correct datatypes from the dataset, so only the names for the columns are required.

Alternatively, data can also be read directly into DataFrames, for instance by using `pandas` or `parquet`.

In [ ]:
df = spark.createDataFrame(data, ['gender', 'age'])
print("The structure of the dataframe is {}".format(df))
#show the result of the dataframe
df.show()
print("the number of rows in the dataframe is {}".format(df.count()))

Different from RDDs, the columns are labelled which allows for clear code.

In [ ]:
df.filter(df.gender == 'M').show()

In [ ]:
df.groupBy(df.gender).count().show()

In [ ]:
df.groupBy(df.gender).max("age").show()

In [ ]:
from pyspark.sql.functions import *
df1 = df.groupBy(df.gender)
df1.agg(sum("age").alias("total"), min("age"), max("age")).show()

For some file formats (e.g. .csv), there are also readers that can read the data directly as DataFrame. In this example, we will use a dataset with airline on-time statistics and delay causes. The spark.csv reader can use the column labels that are in the file. The option inferSchema means that datatypes are correctly inferred. This is actually an expensive operation for large data volumes, alternatively you can manually specify the schema, but in this tutorial, we will stick with inferSchema. 

In [ ]:
filename = '../data/2008.csv.bz2'
if not os.path.exists(filename):
    import urllib.request
    urllib.request.urlretrieve ("http://stat-computing.org/dataexpo/2009/2008.csv.bz2", \
                                filename)

In [ ]:
f = sqlContext.read.format("com.databricks.spark.csv").\
    options(header="true", inferSchema = "true").load(filename)
f.printSchema()

Similar to RDD's, we can perform a `count` on a DataFrame to get the number of row. Notice that this already may take some time on small computers.

In [ ]:
f.count()

Now we have a dataframe, we would like to compute the the average delays for departure and arrival per flight number. 

However, notice that inferschema sometimes guesses wrong: e.g. DepDelay and ArrDelay should be double instead of String. 

Also compute these statistics efficiently, we do not need the entire dataframe, but just a few columns. If we do not leave out the other columns, much more data is processed than necessary.

In [ ]:
from pyspark.sql.types import *

delays = f.select(f.FlightNum, f.DepDelay.cast(DoubleType()),
                  f.ArrDelay.cast(DoubleType()))
delays.printSchema()

Internally, every row in de DataFrame is modelled as a *Row* object that contains a dictionary of values.

In [ ]:
delays.take(5)

And together with the schema, show presents the data as a readable table.

In [ ]:
delays.show(5)

We can reduce the columns in the dataframe by using `select` with a list of columns. A column can either be addressed by its name, or as a property of the dataframe. Reducing the columns in a DataFrame may improve efficiency, since less data is processed.

In [ ]:
delays.printSchema()
delays.show(5)

The API for DataFrame is more similar to SQL than to the API on RDD's. For example, you cannot directly submit lambda functions to process elements, However, the DataFrame does support lazy expressions.

In [ ]:
delays.filter(delays.FlightNum == 335).show(5)

The `select()` transformation can be used to apply expressions to columns, and `alias()` to name the resulting columns. 

In [ ]:
delays.select(f.FlightNum, (delays.ArrDelay - delays.DepDelay).alias('diff')).show(5)

Spark DataFrames also have an SQL like syntax for expressions. Instead of `delays.DepDelay` we can also give the column name `'DepDelay'` which is resolved by looking up the column with that name in the DataFrame. Spark creates new column objects when a new DataFrame is created, and these columns do not always correspond to the original columns. In the example below the groupBy causes new columns to be created, and therefore `delays.DepDelay` would not work and `'DepDelays'` does.

In [ ]:
delays.groupBy( delays.FlightNum).avg( 'DepDelay', 'ArrDelay').show(5)

To perform more complex aggregation functions you can call the generic `agg` tranformation on a DataFrame. Additionally, the `sql.functions` libraray supplies standard functions you can use, e.g. sum, max, count, mean.

In [ ]:
from pyspark.sql.functions import *

fg = delays.groupBy(delays.FlightNum)
fg = fg.agg(mean(delays.ArrDelay - delays.DepDelay), mean("ArrDelay"))
fg.show(5)

Although functions cannot be used by mapping them to elements in the same way as with RDD's, we can create a User Defined Function (UDF) based on a function and use that. For this, `udf` must be imported from `spark.sql.functions` which we already did above.

One catch with udf's is that they return a String by default, converting the output value in the process. We can prevent the `udf` from returning a String by specifying a return type.

NB Spark also has User Defined Aggregate Functions (UDAF), but currently these cannot be easily used from Python. 

In [ ]:
diff = udf(lambda x, y: x - y, DoubleType())

# filter out empty cells
delaysnn = delays.filter(delays.ArrDelay.isNotNull()).filter(delays.DepDelay.isNotNull())
delaysnn.select(delays.FlightNum, diff(delays.ArrDelay, delays.DepDelay)).show(5)


## DataFrame to RDD ##

For some operations, RDD provides a more convenient API than DataFrames. For example, to take the 5 longest departure delays per flights is very hard to do with DataFrames, since there is no standard function in the DataFrame API and no way to group and flatten data. However, we can also use RDD functions by simply converting a DataFrame to an RDD.

Working with RDD's becomes easier when the data has a (key, value) structure. We can prepare that by selecting just the flight number and departure delay. We need to filter out the empty cells. Converting the DataFrame to RDD is as simple as accessing the rdd property of the DataFrame. When we inspect the RDD format, we would see that every element is a DataFrame `Row` object, however, these work just the same as any other tuple. We group the by the key, and can then sort the departure delays per group and take the last 5. We can also flatten the structure so that we get single (FligtNum, DepDelay) elements in one go by using `flatMapValues` instead of `mapValues`.

In [ ]:
depdelays = delays.select(delays.FlightNum, delays.DepDelay)
depdelays = depdelays.filter(depdelays.DepDelay.isNotNull())
delaysrdd = depdelays.rdd
delaysrdd = delaysrdd.groupByKey().\
                      mapValues(lambda x : sorted(x)[-5:])
delaysrdd.take(5)


## caching

When you need intermediate results more often, you can cache them to get faster access. You will experience this when you repeat the show() command several times, the first time is slow, after that it is fast.

In [ ]:
fg.cache()

In [ ]:
fg.show()